In [1]:
! pip install matplotlib numpy opencv-python scikit-image scikit-learn openai gym

In [2]:
import torch
from sam2.build_sam import build_sam2
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator
# from segment_anything import SamAutomaticMaskGenerator, sam_model_registry
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
import cv2
import io
import base64
import os
import json
import re
from openai import AzureOpenAI

In [3]:
sam2_checkpoint = "../checkpoints/sam2.1_hiera_large.pt"
model_cfg = "configs/sam2.1/sam2.1_hiera_l.yaml"
device = "cuda" if torch.cuda.is_available() else "cpu"
sam_model = build_sam2(model_cfg, sam2_checkpoint, device=device, apply_postprocessing=False)

In [4]:
# Authorization token
auth_token = "eyJhbGciOiJSUzI1NiIsIng1dSI6Imltc19uYTEtc3RnMS1rZXktYXQtMS5jZXIiLCJraWQiOiJpbXNfbmExLXN0ZzEta2V5LWF0LTEiLCJpdHQiOiJhdCJ9.eyJpZCI6IjE3MzY4Nzc4NDY4MjFfZTBmYjVmMmYtZWVmYy00ZGVmLWI2ZTktODkwNmQzNGQyZDU0X3V3MiIsInR5cGUiOiJhY2Nlc3NfdG9rZW4iLCJjbGllbnRfaWQiOiJJRENsb3VkU2VydmljZTIiLCJ1c2VyX2lkIjoiSURDbG91ZFNlcnZpY2UyQEFkb2JlSUQiLCJhcyI6Imltcy1uYTEtc3RnMSIsImFhX2lkIjoiSURDbG91ZFNlcnZpY2UyQEFkb2JlSUQiLCJjdHAiOjAsInBhYyI6IklEQ2xvdWRTZXJ2aWNlMl9zdGcyIiwicnRpZCI6IjE3MzY4Nzc4NDY4MjFfZTk1NWFiMmUtMmZhYy00NTdhLTg4YmUtZGRhNmQ0NjEzMTgzX3V3MiIsIm1vaSI6ImQyNWYxZmM2IiwicnRlYSI6IjE3MzgwODc0NDY4MjEiLCJleHBpcmVzX2luIjoiODY0MDAwMDAiLCJzY29wZSI6InN5c3RlbSxvcGVuaWQsYWRkaXRpb25hbF9pbmZvLm93bmVyT3JnLGFkbWluX21hbmFnZV93b3JrZmxvd3MsaW5kZXNpZ25fc2VydmljZXMiLCJjcmVhdGVkX2F0IjoiMTczNjg3Nzg0NjgyMSJ9.OkXCRGjzOigH6l5MsiXJQZMs5-SnU4o12nTC_TlhXxuN0j26EwfL2AA2w-LKCPa46TECH8ni2VNwChY9hCUaxDy3TglfGtT5_LVjdiydxDtn2XFOljLLL0KMMYflUWEy9GcoJgjMw86o1_LTxL9eCqNroeXHoOdowM_Bhfg1NX1S2rpmU1An4qtYS25nSU2yX1YkS_Uw04LCrELD4d3WnWjEJKI4elHsVhkoG8seE64JA2LPCks2Q2EXy8PYaqv064Z2OQPOq8zV00-J4GCaLI51FUZab13j5kTIUHqVZ32fY4Da9ydh2ipgFhfBhCwMQF5H4rZ9b4WVrNL7L7zidQ"

# Azure OpenAI API key
azure_openai_api_key = "FkTtgcdo82dJwR4h23hM1bZ9cahYfHeOkH4rFLIfxqOmLWr7unlkJQQJ99BAACYeBjFXJ3w3AAABACOGacz3"  # Replace with your Azure OpenAI API key

# Azure OpenAI API endpoint
azure_openai_api_endpoint = "https://gpt-ml.openai.azure.com/"

# Azure OpenAI model name = "gpt-4o"
azure_openai_model_name = "gpt-4o"

# Azure OpenAI model version = "2024-10-01-preview"
azure_openai_model_version = "2024-10-01-preview"

In [5]:
# Initialize the Azure OpenAI client
azure_openai_client = AzureOpenAI(  
    azure_endpoint=azure_openai_api_endpoint,  
    api_key=azure_openai_api_key,  
    api_version=azure_openai_model_version,  
)

In [6]:
# Async function to encode a local image into data URL
async def local_image_bytes_to_data_url(image_bytes):
    # Guess the MIME type of the image based on the file extension
    mime_type = Image.open(io.BytesIO(image_bytes)).get_format_mimetype()
    if mime_type is None:
        mime_type = 'application/octet-stream'  # Default MIME type if none is found

    base64_encoded_data = base64.b64encode(image_bytes).decode('utf-8')

    # Construct the data URL
    return f"data:{mime_type};base64,{base64_encoded_data}"

In [7]:
# async method for extracting json from text json response
async def extract_json_from_response(extracted_text_resp):
    try:
        response_json = json.loads(extracted_text_resp)
        return response_json
    except ValueError:
        print(f"extracted_text_resp is not a valid JSON string")
        pass

    # Extract the JSON from the response
    pattern = r'(.*?)```json(.*?)```(.*)'

    # Search for the pattern in the input string using re.DOTALL to include newlines
    match = re.search(pattern, extracted_text_resp, re.DOTALL)

    if match:
        # Extract the JSON content and strip leading/trailing whitespace
        json_str_resp = match.group(2).strip()
        response_json = json.loads(json_str_resp)
        # print(json.dumps(response_json, indent=4))
        return response_json
    else:
        # Raise an error if the pattern is not found
        raise ValueError("JSON content not found in the input string.")
    return None

In [8]:
# async method for detecting background elements in an image with gpt-4-vision api
async def detect_background_elements(image_data_url):
    # Sample output JSON for background elements
    output_json = {
        "image_size": {
            "width": "<Width of the bounding box>",
            "height": "<Height of the bounding box>"
        },
        "background_elements": [
            {
                "type": "filled-rectangle",
                "bounding_box": {
                    "x": "<X-coordinate of the top-left corner>",
                    "y": "<Y-coordinate of the top-left corner>",
                    "width": "<Width of the bounding box>",
                    "height": "<Height of the bounding box>"
                },
                "fillColor": "#87CEEB"
            }
        ]
    }

    background_extraction_prompt = """
    **Task Description:**  
    Analyze the provided image and extract all background details as structured data. 
    The goal is to create a JSON representation of the background that can be used to redraw it on a white canvas. 
    Focus on identifying backgrounds, patterns, shapes, colors, and gradients, ignoring any foreground elements.

    **Steps:**  
    1. Examine the input image and locate all background components.  
    2. Distinguish between different types of background elements (solid colors, gradients, shapes, patterns).
    3. background element should not contain any text related object. 
    3. For each element, record its type, bounding box(x, y, width, height), color(s), and any other relevant attributes.  
    4. Compile these details into a structured JSON object along with the image size that can be used to reconstruct the background on a blank canvas.
    5. Provide only the structured JSON representation of the background, without any extra details or text information.

    **Input Image:** 

    Attached in image_url

    **Expected Output Example:**
    Respond only in the following JSON format:

    {output_json}
    """

    user_message = {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": background_extraction_prompt
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": image_data_url
                }
            }
        ]
    }

    chat_prompt = [
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": "You are an AI assistant that helps people find information."
                }
            ]
        },
        user_message
    ]

    # Generate the completion  
    # make a request to the gpt-4o api
    completion = azure_openai_client.chat.completions.create(  
        model=azure_openai_model_name,  
        messages=chat_prompt,  
        max_tokens=2800,  
        temperature=0.1,  
        top_p=0.9,  
        frequency_penalty=0,  
        presence_penalty=0,  
        stop=None,  
        stream=False
    )

    response = (completion.to_json())
    extracted_text_resp = json.loads(response)['choices'][0]['message']['content']
    print(f"Extracted text response for background elements: {extracted_text_resp}")
    background_elements_json = await extract_json_from_response(extracted_text_resp)
    print(f"Background elements JSON: {background_elements_json}")
    return background_elements_json

In [9]:
def adjust_background_elements(bg_elements, new_width, new_height):
    curr_img_size = bg_elements["image_size"]
    original_width = curr_img_size["width"]
    original_height = curr_img_size["height"]   
    width_ratio = new_width / original_width
    height_ratio = new_height / original_height

    adjust_background_elements = {
        "image_size": {
            "width": new_width,
            "height": new_height
        },
        "background_elements": []
    }

    adjusted_elements = []
    for element in bg_elements["background_elements"]:
        print(f"Element: {element}")
        if "color" not in element:
            continue
        adjusted_bounding_box = {
            "x": element["bounding_box"]["x"] * width_ratio,
            "y": element["bounding_box"]["y"] * height_ratio,
            "width": element["bounding_box"]["width"] * width_ratio,
            "height": element["bounding_box"]["height"] * height_ratio
        }
        adjusted_element = {
            "color": element["color"],
            "type": element["type"],
            "bounding_box": adjusted_bounding_box
        }
        adjusted_elements.append(adjusted_element)
    
    adjust_background_elements["background_elements"] = adjusted_elements
    return adjust_background_elements

In [10]:
# function for drawing background elements on a canvas
def draw_background_elements(canvas, background_elements, image_size):
    # Create a new image with the specified size
    canvas = Image.new("RGB", (image_size["width"], image_size["height"]), (255, 255, 255))
    # Create a drawing object
    draw = ImageDraw.Draw(canvas)
    print(f"background_elements: {background_elements}")
    for element in background_elements["background_elements"]:
        print(f"Element: {element}")
        print(f"Element type: {element["type"]}")
        if element["type"] in ["filled-rectangle", "solid_color", "rectangle"]:
            # Extract the position, size, and color of the filled rectangle
            x = element["bounding_box"]["x"]
            y = element["bounding_box"]["y"]
            width = element["bounding_box"]["width"]
            height = element["bounding_box"]["height"]
            fill_color = element["color"]
            draw.rectangle([x, y, x + width, y + height], fill=fill_color)
    canvas.show()
    return canvas

In [11]:
# async method for extracting json from text json response
async def extract_json_from_response(extracted_text_resp):
    try:
        response_json = json.loads(extracted_text_resp)
        return response_json
    except ValueError:
        print(f"extracted_text_resp is not a valid JSON string")
        pass

    # Extract the JSON from the response
    pattern = r'(.*?)```json(.*?)```(.*)'

    # Search for the pattern in the input string using re.DOTALL to include newlines
    match = re.search(pattern, extracted_text_resp, re.DOTALL)

    if match:
        # Extract the JSON content and strip leading/trailing whitespace
        json_str_resp = match.group(2).strip()
        response_json = json.loads(json_str_resp)
        # print(json.dumps(response_json, indent=4))
        return response_json
    else:
        # Raise an error if the pattern is not found
        raise ValueError("JSON content not found in the input string.")
    return None

In [12]:
def paste_objects_on_blank_canvas(image, masks):
    """
    Cut out each object defined by SAM2 masks and paste onto a blank canvas.
    
    Args:
        image (np.ndarray): Original image in BGR or RGB (H x W x 3).
        masks (list[dict]): List of masks from SAM2 with 'segmentation' and 'bbox' keys.
    
    Returns:
        np.ndarray: Blank canvas with all segmented objects pasted.
    """
    # Prepare a blank canvas of the same dimensions and type as 'image'
    height, width = image.shape[:2]
    blank_canvas = np.zeros((height, width, 3), dtype=np.uint8)
    
    for mask_info in masks:
        # Segmentation mask (boolean array the same size as the image)
        seg = mask_info['segmentation']
        
        # BBox is in XYWH format. Convert to integer if needed.
        x, y, w, h = map(int, mask_info['bbox'])
        x2, y2 = x + w, y + h
        
        # Crop out the object region from the original image
        object_region = image[y:y2, x:x2]
        
        # For convenience, get the corresponding region in the segmentation mask
        seg_region = seg[y:y2, x:x2]
        
        # "Paste" the object onto the blank canvas where the mask is True
        blank_canvas[y:y2, x:x2][seg_region] = object_region[seg_region]
    
    return blank_canvas

In [13]:
def paste_objects_on_background_canvas(image, masks, background_image):
    """
    Cut out each object defined by SAM2 masks and paste onto a background canvas.
    
    Args:
        image (np.ndarray): Original image in BGR or RGB (H x W x 3).
        masks (list[dict]): List of masks from SAM2 with 'segmentation' and 'bbox' keys.
        background_image (PIL.Image.Image or np.ndarray): Background image in BGR or RGB (H x W x 3).
    
    Returns:
        np.ndarray: Background canvas with all segmented objects pasted.
    """
    # Convert background_image to a NumPy array if it's a PIL Image
    if isinstance(background_image, Image.Image):
        background_image = np.array(background_image)
    
    # Ensure the background image has the same dimensions as the original image
    height, width = image.shape[:2]

    background_canvas = None
    
    if background_image.shape[:2] != (height, width):
        background_canvas = cv2.resize(background_image, (width, height))
    else:
        background_canvas = background_image.copy()
    
    for mask_info in masks:
        # Segmentation mask (boolean array the same size as the image)
        seg = mask_info['segmentation']
        
        # BBox is in XYWH format. Convert to integer if needed.
        x, y, w, h = map(int, mask_info['bbox'])
        x2, y2 = x + w, y + h
        
        # Crop out the object region from the original image
        object_region = image[y:y2, x:x2]
        
        # For convenience, get the corresponding region in the segmentation mask
        seg_region = seg[y:y2, x:x2]
        
        # "Paste" the object onto the background canvas where the mask is True
        background_canvas[y:y2, x:x2][seg_region] = object_region[seg_region]
    
    return background_canvas

In [14]:
# # 1. Load an example image (modify path to your own image)
# image_path = "../inputs/surf_school_test_image.png"
# original_image = cv2.imread(image_path)  
# # Ensure 'original_image' is loaded. If None, check the file path.

# # For demonstration, we'll just return the image size
# width, height, _ = original_image.shape
# # Convert the image to bytes
# _, img_encoded = cv2.imencode('.png', original_image)
# img_byte_arr = img_encoded.tobytes()
# # exrtact the image data url from the image byte array
# image_data_url = await local_image_bytes_to_data_url(img_byte_arr)
# # print(f"Image data url:  {image_data_url}")

# # exrtact the background elements from the image
# background_elements = await detect_background_elements(image_data_url)
# print(f"Background elements: {background_elements}")

# # Adjust the background elements to match the new canvas size
# adjusted_background_elements = adjust_background_elements(background_elements, width, height)

# # Draw the background elements on a new canvas
# adjusted_background_elements_img_size  = adjusted_background_elements["image_size"]
# (new_width, new_height) = (adjusted_background_elements_img_size["width"], adjusted_background_elements_img_size["height"])
# print(f"New canvas size: {new_width} x {new_height}")
# background_canvas = draw_background_elements(None, adjusted_background_elements, {"width": new_width, "height": new_height})

# # 2. Generate masks using SAM2
# # For illustration, assume 'mask_generator_2' is already defined/configured.
# mask_generator = SAM2AutomaticMaskGenerator(sam_model)
# # mask_generator_2 = SAM2AutomaticMaskGenerator(
# #     model=sam_model,
# #     points_per_side=128,  # Increased for finer granularity in object detection.
# #     points_per_batch=256,  # Increased to handle more points efficiently.
# #     pred_iou_thresh=0.6,  # Slightly lowered to include more objects with less strict IoU.
# #     stability_score_thresh=0.88,  # Balanced for both stability and inclusion.
# #     stability_score_offset=0.6,  # Slightly reduced for broader stability score acceptance.
# #     crop_n_layers=2,  # More cropping layers to capture complex hierarchies.
# #     box_nms_thresh=0.6,  # Reduced to allow slightly overlapping detections.
# #     crop_n_points_downscale_factor=1,  # No downscaling for precise object segmentation.
# #     min_mask_region_area=10.0,  # Reduced to capture smaller objects like text or small graphics.
# #     use_m2m=True,  # Enable mask-to-mask refinement for higher-quality results.
# # )
# # masks = mask_generator_2.generate(original_image)
# masks = mask_generator.generate(original_image)

# # 3. Paste all detected objects onto a blank canvas
# canvas_with_objects = paste_objects_on_blank_canvas(original_image, masks)

# # draw objects on the background canvas
# background_canvas_with_objects = paste_objects_on_background_canvas(original_image, masks, background_canvas)

# # 4. Show the result
# # Option A: Use OpenCV (BGR format)
# # cv2.imshow("Canvas with Objects", canvas_with_objects)
# # cv2.waitKey(0)
# # cv2.destroyAllWindows()

# # oringal image
# cv2.imshow("original_image", original_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
# # plt.imshow(original_image, cv2.COLOR_BGR2RGB)
# # plt.title("Original image")
# # plt.axis("off")
# # plt.show()

# # Option B: Use Matplotlib (assumes RGB format)
# # Convert from BGR to RGB for correct color rendering in Matplotlib
# # canvas_rgb = cv02.cvtColor(canvas_with_objects, cv2.COLOR_BGR2RGB)
# cv2.imshow("canvas with objects", canvas_with_objects)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# # Convert from BGR to RGB for correct color rendering in Matplotlib
# # background_canvas_rgb = cv2.cvtColor(background_canvas_with_objects, cv2.COLOR_BGR2RGB)
# cv2.imshow("background canvas with objects", background_canvas_with_objects)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [15]:
# 1. Load an example image (modify path to your own image)
image_path = "../inputs/candel.png"
original_image = cv2.imread(image_path)  
# Ensure 'original_image' is loaded. If None, check the file path.

# For demonstration, we'll just return the image size
width, height, _ = original_image.shape
# Convert the image to bytes
_, img_encoded = cv2.imencode('.png', original_image)
img_byte_arr = img_encoded.tobytes()
# exrtact the image data url from the image byte array
image_data_url = await local_image_bytes_to_data_url(img_byte_arr)
# print(f"Image data url:  {image_data_url}")

# exrtact the background elements from the image
background_elements = await detect_background_elements(image_data_url)
print(f"Background elements: {background_elements}")

# Adjust the background elements to match the new canvas size
adjusted_background_elements = adjust_background_elements(background_elements, width, height)

# Draw the background elements on a new canvas
adjusted_background_elements_img_size  = adjusted_background_elements["image_size"]
(new_width, new_height) = (adjusted_background_elements_img_size["width"], adjusted_background_elements_img_size["height"])
print(f"New canvas size: {new_width} x {new_height}")
background_canvas = draw_background_elements(None, adjusted_background_elements, {"width": new_width, "height": new_height})

# 2. Generate masks using SAM2
# For illustration, assume 'mask_generator_2' is already defined/configured.
# mask_generator = SAM2AutomaticMaskGenerator(sam_model)
mask_generator_2 = SAM2AutomaticMaskGenerator(
    model=sam_model,
    points_per_side=128,  # Increased for finer granularity in object detection.
    points_per_batch=256,  # Increased to handle more points efficiently.
    min_mask_region_area=10.0,  # Reduced to capture smaller objects like text or small graphics.
)
masks = mask_generator_2.generate(original_image)
# masks = mask_generator.generate(original_image)

# 3. Paste all detected objects onto a blank canvas
canvas_with_objects = paste_objects_on_blank_canvas(original_image, masks)

# draw objects on the background canvas
background_canvas_with_objects = paste_objects_on_background_canvas(original_image, masks, background_canvas)

# 4. Show the result
# Option A: Use OpenCV (BGR format)
# cv2.imshow("Canvas with Objects", canvas_with_objects)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# oringal image
display(original_image)
# cv2.imshow("original_image", original_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
# plt.imshow(original_image, cv2.COLOR_BGR2RGB)
# plt.title("Original image")
# plt.axis("off")
# plt.show()

# Option B: Use Matplotlib (assumes RGB format)
# Convert from BGR to RGB for correct color rendering in Matplotlib
# canvas_rgb = cv02.cvtColor(canvas_with_objects, cv2.COLOR_BGR2RGB)
cv2.imshow("canvas with objects", canvas_with_objects)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Convert from BGR to RGB for correct color rendering in Matplotlib
# background_canvas_rgb = cv2.cvtColor(background_canvas_with_objects, cv2.COLOR_BGR2RGB)
cv2.imshow("background canvas with objects", background_canvas_with_objects)
cv2.waitKey(0)
cv2.destroyAllWindows()

Extracted text response for background elements: ```json
{
  "image_size": {
    "width": 1085,
    "height": 768
  },
  "background_elements": [
    {
      "type": "solid_color",
      "bounding_box": {
        "x": 0,
        "y": 0,
        "width": 1085,
        "height": 768
      },
      "color": "#2F3E46"
    },
    {
      "type": "shape",
      "bounding_box": {
        "x": 492,
        "y": 400,
        "width": 100,
        "height": 50
      },
      "color": "#FFD166",
      "shape_type": "semi_circle"
    },
    {
      "type": "horizontal_line",
      "bounding_box": {
        "x": 100,
        "y": 600,
        "width": 885,
        "height": 5
      },
      "color": "#FFD166"
    },
    {
      "type": "horizontal_line",
      "bounding_box": {
        "x": 100,
        "y": 650,
        "width": 885,
        "height": 5
      },
      "color": "#FFD166"
    }
  ]
}
```
extracted_text_resp is not a valid JSON string
Background elements JSON: {'image_size': {'width'

/Users/anupamgadobe.com/Work/sam2/sam2/sam2_image_predictor.py:431: UserWarning: cannot import name '_C' from 'sam2' (/Users/anupamgadobe.com/Work/sam2/sam2/__init__.py)

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/sam2/blob/main/INSTALL.md).
  masks = self._transforms.postprocess_masks(


: 